In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
import numpy as np
import pandas as pd
from glob import glob

In [ ]:
import h5py
import nilearn as nl
from nilearn import image, plotting

### Load templates

In [ ]:
# Load brain mask
mask = nl.image.load_img('templates/fMRI_mask.nii')

In [ ]:
# Download GM, WM and CSF probability maps from ICBM 2009c asymmetric template
# From: http://www.bic.mni.mcgill.ca/ServicesAtlases/ICBM152NLin2009
template_gm = nl.image.load_img('templates/mni_icbm152_nlin_asym_09c/mni_icbm152_gm_tal_nlin_asym_09c.nii')
template_gm = image.resample_to_img(template_gm, mask)
pve_gm = image.math_img('img * mask', img=template_gm, mask=mask)

template_wm = nl.image.load_img('templates/mni_icbm152_nlin_asym_09c/mni_icbm152_wm_tal_nlin_asym_09c.nii')
template_wm = image.resample_to_img(template_wm, mask)
pve_wm = image.math_img('img * mask', img=template_wm, mask=mask)

template_csf = nl.image.load_img('templates/mni_icbm152_nlin_asym_09c/mni_icbm152_csf_tal_nlin_asym_09c.nii')
template_csf = image.resample_to_img(template_csf, mask)
pve_csf = image.math_img('img * mask', img=template_csf, mask=mask)

In [ ]:
# Create pve mask
pve_concat = image.concat_imgs([pve_gm, pve_wm, pve_csf])
pve_mask = image.math_img('np.sum(img, axis=-1)>0.5', img=pve_concat)

In [ ]:
# Find pve binary masks per tissue
pve_argmax = image.math_img('np.argmax(img, axis=-1) * mask', img=pve_concat, mask=mask)
pve_mask_gm, pve_mask_wm, pve_mask_csf = [image.math_img('(img==%d)*mask' % i, img=pve_argmax, mask=mask) for i in range(3)]

### Create kaggle datasets

In [ ]:
def read_img(filename, mask):
    with h5py.File(filename, 'r') as f:
        data = np.array(f['SM_feature'], dtype='float32')

    # It's necessary to reorient the axes, since h5py flips axis order
    data = np.moveaxis(data, [0, 1, 2, 3],
                             [3, 2, 1, 0])

    img = nl.image.new_img_like(mask, data, affine=mask.affine, copy_header=True)
    return img

    # Rewrite mat file to compressed NIfTI
    for fname in tqdm(sorted(glob('fMRI_tr*/*.mat'))):
        new_filename = fname.replace('.mat', '.nii.gz')
        read_img(fname, mask).to_filename(new_filename)

    # Rewrite mat file to compressed NIfTI
    for fname in tqdm(sorted(glob('fMRI_te*/*.mat'))):
        new_filename = fname.replace('.mat', '.nii.gz')
        read_img(fname, mask).to_filename(new_filename)

## Explor NIfTI files

In [ ]:
from nilearn import image, plotting, masking
from nilearn.regions import connected_regions

In [ ]:
filenames = sorted(glob('fMRI_train/*.nii.gz'))[::10]
print(len(filenames))

In [ ]:
# Create the mean image for a given component
def get_mean_component(filenames, comp_ID=0):
    mean = image.math_img('img * 0', img=mask)
    for f in tqdm(filenames):
        img = image.load_img(f).slicer[..., comp_ID]
        mean = image.math_img('mean + img', mean=mean, img=img)
    mean = image.math_img('img / %f' % len(filenames), img=mean)
    return mean

In [ ]:
# Extract the mean images
for idx in range(53):
    mean = get_mean_component(filenames, comp_ID=idx)
    mean.to_filename('fMRI_maps/mean_%02d.nii.gz' % (idx + 1))

In [ ]:
# Extract the mean images
for idx in range(5):
    img = image.load_img('fMRI_maps/mean_%02d.nii.gz' % (idx + 1))
    data = img.get_fdata()
    threshold = np.percentile(data[data!=0], 99)
    img_thr = image.threshold_img(img, threshold=threshold)
    img_regions = image.mean_img(connected_regions(img_thr, min_region_size=4000)[0])
    plotting.plot_glass_brain(img_regions, black_bg=True, display_mode='lyrz',
                              title='mean_%02d' % (idx + 1))
    plt.show()

## Extract relevant features from regions

In [ ]:
ls datasets/*raw_train_*csv

In [ ]:
n_train = len(glob('datasets/*raw_train_*csv'))
n_train

In [ ]:
# Collect value metrics from images
train_files = sorted(glob('fMRI_train/*.nii.gz'))

for idx in range(n_train, 53):

    # Load mean image
    mean = image.load_img('fMRI_maps/mean_%02d.nii.gz' % (idx + 1))
    data_mean = mean.get_fdata()[mask.get_fdata()>0]
    
    # Compute binary mask for region
    mask_region = data_mean > np.percentile(data_mean, 99)

    # Store results in results file
    results = {}
    
    for t in tqdm(train_files):

        try:
            # Get file name
            t_id = t.split('/')[1].split('.')[0]

            # Load current volume
            img = image.index_img(t, idx)

            # Only extract data values from within mask
            data_img = img.get_fdata()[mask.get_fdata()>0]

            # Collect correlation coefficient to mean image
            corr_coef = np.corrcoef(data_img, data_mean)[0, 1]

            # Collect euclidean distance to mean image
            euclide_whole = np.linalg.norm(np.subtract(data_img, data_mean))
            euclide_region = np.linalg.norm(np.subtract(data_img[mask_region], data_mean[mask_region]))

            # Collect percentiles from whole image and region
            perc_to_check_r = [0.1, 1, 5, 50, 95, 99, 99.9]
            percentiles_whole = [np.percentile(data_img, p) for p in perc_to_check_r]
            percentiles_region = [np.percentile(data_img[mask_region], p) for p in perc_to_check_r]

            # Extract volume data from within PVE masks
            pve_masked_gm_values = img.get_fdata()[pve_mask_gm.get_fdata().astype('bool')]
            pve_masked_wm_values = img.get_fdata()[pve_mask_wm.get_fdata().astype('bool')]
            pve_masked_csf_values = img.get_fdata()[pve_mask_csf.get_fdata().astype('bool')]

            # Collect percentiles from tissue masks
            perc_to_check_t = [1, 5, 50, 95, 99]
            percentiles_gm = [np.percentile(pve_masked_gm_values, p) for p in perc_to_check_t]
            percentiles_wm = [np.percentile(pve_masked_wm_values, p) for p in perc_to_check_t]
            percentiles_csf = [np.percentile(pve_masked_csf_values, p) for p in perc_to_check_t]

            # Compute smoothness to original image difference
            smoothness = np.linalg.norm(image.math_img(
                'img-smooth', img=img, smooth=image.smooth_img(
                    img, 6)).get_fdata()[mask.get_fdata()>0])

            # Compute coefficient of joint variation (CJV) within GM and WM
            cjv = (pve_masked_wm_values.std() + pve_masked_gm_values.std()) / \
                   np.abs(pve_masked_wm_values.mean() - pve_masked_gm_values.mean())

            # Compute signal to noise ratio
            snr_gm = pve_masked_gm_values.mean() / (pve_masked_gm_values.std() * np.sqrt(len(pve_masked_gm_values)/(len(pve_masked_gm_values)-1)))
            snr_wm = pve_masked_wm_values.mean() / (pve_masked_wm_values.std() * np.sqrt(len(pve_masked_wm_values)/(len(pve_masked_wm_values)-1)))
            snr_csf = pve_masked_csf_values.mean() / (pve_masked_csf_values.std() * np.sqrt(len(pve_masked_csf_values)/(len(pve_masked_csf_values)-1)))

            # Compute wm2max values
            muWM = pve_masked_wm_values.mean()
            wm2max_gm = muWM/np.percentile(pve_masked_gm_values, 99.95)
            wm2max_wm = muWM/np.percentile(pve_masked_wm_values, 99.95)
            wm2max_csf = muWM/np.percentile(pve_masked_csf_values, 99.95)

            # Collect standard deviation from whole image and region
            whole_std = data_img.std()
            region_std = data_img[mask_region].std()

            results[t_id] = [
                t_id, corr_coef, euclide_whole, euclide_region, whole_std, region_std,
                *percentiles_whole, *percentiles_region,
                *percentiles_gm, *percentiles_wm, *percentiles_csf,
                smoothness, cjv,
                snr_gm, snr_wm, snr_csf,
                wm2max_gm, wm2max_wm, wm2max_csf,
                ]
        except:
            print(t)

    # Store result in CSV file
    df_results = pd.DataFrame(results).T
    df_results.columns = ['Id', 'corr_coef',
                          'euclide_whole', 'euclide_region',
                          'whole_std', 'region_std',
                          *['perc-%s_whole' % p for p in perc_to_check_r],
                          *['perc-%s_region' % p for p in perc_to_check_r],
                          *['perc-%s_gm' % p for p in perc_to_check_t],
                          *['perc-%s_wm' % p for p in perc_to_check_t],
                          *['perc-%s_csf' % p for p in perc_to_check_t],
                          'smoothness',
                          'cjv',
                          'snr_gm', 'snr_wm', 'snr_csf',
                          'wm2max_gm', 'wm2max_wm', 'wm2max_csf']
    df_results.to_csv('datasets/brain_values_raw_train_%02d.csv' % (idx + 1), index=False)

In [ ]:
ls datasets/*values_raw_test*csv

In [ ]:
n_test = len(glob('datasets/*raw_test_*csv'))
n_test

In [ ]:
# Collect results
results = {}

test_files = sorted(glob('fMRI_test/*.nii.gz'))

for idx in range(n_test, 53):

    # Load mean image
    mean = image.load_img('fMRI_maps/mean_%02d.nii.gz' % (idx + 1))
    data_mean = mean.get_fdata()[mask.get_fdata()>0]
    
    # Compute binary mask for region
    mask_region = data_mean > np.percentile(data_mean, 99)

    # Store results in results file
    results = {}
    
    for t in tqdm(test_files):

        try:
            # Get file name
            t_id = t.split('/')[1].split('.')[0]

            # Load current volume
            img = image.index_img(t, idx)

            # Only extract data values from within mask
            data_img = img.get_fdata()[mask.get_fdata()>0]

            # Collect correlation coefficient to mean image
            corr_coef = np.corrcoef(data_img, data_mean)[0, 1]

            # Collect euclidean distance to mean image
            euclide_whole = np.linalg.norm(np.subtract(data_img, data_mean))
            euclide_region = np.linalg.norm(np.subtract(data_img[mask_region], data_mean[mask_region]))

            # Collect percentiles from whole image and region
            perc_to_check_r = [0.1, 1, 5, 50, 95, 99, 99.9]
            percentiles_whole = [np.percentile(data_img, p) for p in perc_to_check_r]
            percentiles_region = [np.percentile(data_img[mask_region], p) for p in perc_to_check_r]

            # Extract volume data from within PVE masks
            pve_masked_gm_values = img.get_fdata()[pve_mask_gm.get_fdata().astype('bool')]
            pve_masked_wm_values = img.get_fdata()[pve_mask_wm.get_fdata().astype('bool')]
            pve_masked_csf_values = img.get_fdata()[pve_mask_csf.get_fdata().astype('bool')]

            # Collect percentiles from tissue masks
            perc_to_check_t = [1, 5, 50, 95, 99]
            percentiles_gm = [np.percentile(pve_masked_gm_values, p) for p in perc_to_check_t]
            percentiles_wm = [np.percentile(pve_masked_wm_values, p) for p in perc_to_check_t]
            percentiles_csf = [np.percentile(pve_masked_csf_values, p) for p in perc_to_check_t]

            # Compute smoothness to original image difference
            smoothness = np.linalg.norm(image.math_img(
                'img-smooth', img=img, smooth=image.smooth_img(
                    img, 6)).get_fdata()[mask.get_fdata()>0])

            # Compute coefficient of joint variation (CJV) within GM and WM
            cjv = (pve_masked_wm_values.std() + pve_masked_gm_values.std()) / \
                   np.abs(pve_masked_wm_values.mean() - pve_masked_gm_values.mean())

            # Compute signal to noise ratio
            snr_gm = pve_masked_gm_values.mean() / (pve_masked_gm_values.std() * np.sqrt(len(pve_masked_gm_values)/(len(pve_masked_gm_values)-1)))
            snr_wm = pve_masked_wm_values.mean() / (pve_masked_wm_values.std() * np.sqrt(len(pve_masked_wm_values)/(len(pve_masked_wm_values)-1)))
            snr_csf = pve_masked_csf_values.mean() / (pve_masked_csf_values.std() * np.sqrt(len(pve_masked_csf_values)/(len(pve_masked_csf_values)-1)))

            # Compute wm2max values
            muWM = pve_masked_wm_values.mean()
            wm2max_gm = muWM/np.percentile(pve_masked_gm_values, 99.95)
            wm2max_wm = muWM/np.percentile(pve_masked_wm_values, 99.95)
            wm2max_csf = muWM/np.percentile(pve_masked_csf_values, 99.95)

            # Collect standard deviation from whole image and region
            whole_std = data_img.std()
            region_std = data_img[mask_region].std()

            results[t_id] = [
                t_id, corr_coef, euclide_whole, euclide_region, whole_std, region_std,
                *percentiles_whole, *percentiles_region,
                *percentiles_gm, *percentiles_wm, *percentiles_csf,
                smoothness, cjv,
                snr_gm, snr_wm, snr_csf,
                wm2max_gm, wm2max_wm, wm2max_csf,
                ]
        except:
            print(t)

    # Store result in CSV file
    df_results = pd.DataFrame(results).T
    df_results.columns = ['Id', 'corr_coef',
                          'euclide_whole', 'euclide_region',
                          'whole_std', 'region_std',
                          *['perc-%s_whole' % p for p in perc_to_check_r],
                          *['perc-%s_region' % p for p in perc_to_check_r],
                          *['perc-%s_gm' % p for p in perc_to_check_t],
                          *['perc-%s_wm' % p for p in perc_to_check_t],
                          *['perc-%s_csf' % p for p in perc_to_check_t],
                          'smoothness',
                          'cjv',
                          'snr_gm', 'snr_wm', 'snr_csf',
                          'wm2max_gm', 'wm2max_wm', 'wm2max_csf']
    df_results.to_csv('datasets/brain_values_raw_test_%02d.csv' % (idx + 1), index=False)

## Extract relevant features within subjects

In [ ]:
# Collect results
corr_results = {}

# Collect all train files
train_files = sorted(glob('fMRI_train/*.nii.gz'))

for t in tqdm(train_files):

    try:
        # Load mean image
        img = image.load_img(t)
        data = img.get_fdata()[mask.get_fdata()>0]

        t_id = t.split('/')[1].split('.')[0]
        corr_results[t_id] = np.ravel(np.corrcoef(data.T))

    except:
            print(t)

df_corr = pd.DataFrame(corr_results).T
df_corr.columns = ['c%02d_c%02d' % (i + 1, j + 1)
                   for i in range(53) for j in range(53)]

# Only keep upper triangular correlation matrix without diagonal
triangular_mask = np.ravel(np.triu(np.ones((53, 53)), k=1))>0.5
df_corr = df_corr.loc[:, triangular_mask]

# Save everything in CSV file
df_corr.to_csv('datasets/brain_corr_raw_train.csv')
df_corr.head()

In [ ]:
# Collect results
corr_results = {}

# Collect all test files
test_files = sorted(glob('fMRI_test/*.nii.gz'))

for t in tqdm(test_files):

    try:
        # Load mean image
        img = image.load_img(t)
        data = img.get_fdata()[mask.get_fdata()>0]

        t_id = t.split('/')[1].split('.')[0]
        corr_results[t_id] = np.ravel(np.corrcoef(data.T))

    except:
            print(t)

df_corr = pd.DataFrame(corr_results).T
df_corr.columns = ['c%02d_c%02d' % (i + 1, j + 1)
                   for i in range(53) for j in range(53)]

# Only keep upper triangular correlation matrix without diagonal
triangular_mask = np.ravel(np.triu(np.ones((53, 53)), k=1))>0.5
df_corr = df_corr.loc[:, triangular_mask]

# Save everything in CSV file
df_corr.to_csv('datasets/brain_corr_raw_test.csv')
df_corr.head()

# Find region correlating high with target categories

In [ ]:
ls datasets/*brain_vox_raw_test*csv

In [ ]:
n_train = len(glob('datasets/*brain_vox_raw_test_*csv'))
n_train

In [ ]:
from sklearn.model_selection import train_test_split

# Load brain mask
mask = nl.image.load_img('templates/fMRI_mask.nii')

# Collect value metrics from images
train_files = sorted(glob('fMRI_train/*.nii.gz'))
test_files = sorted(glob('fMRI_test/*.nii.gz'))

for idx in range(n_train, 53):
    
    data_collection = []
    tid_collection = []

    for t in tqdm(train_files):

        try:
            # Load current volume and extract data values only from within the mask
            data_img = image.index_img(t, idx).get_fdata()[mask.get_fdata()>0]

            # Store data
            data_collection.append(data_img)

            # Get file name
            t_id = t.split('/')[1].split('.')[0]
            tid_collection.append(t_id)

        except:
            print(t_id)

    data_collection = np.array(data_collection)
    tid_collection = np.array(tid_collection).astype('int')
    
    # To save all relevant voxel_idx
    voxel_to_extract = []

    # Select top correlating voxels in two different splits
    nsteps = 2

    target_column = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']
    
    for t in tqdm(target_column):

        print('Computes the correlation coefficients for: %s' % (t))

        # For every component, randomly reshuffle order of entries
        target = pd.read_csv('train_scores.csv')[['Id', t]].dropna()

        # Find overlap between target ID and brains
        overlap = np.intersect1d(target.Id, tid_collection)
        
        # Keep only overlaps
        target = target[target.Id.isin(overlap)]
        data_target = data_collection[[t in overlap for t in tid_collection]]
        tid_target = tid_collection[[t in overlap for t in tid_collection]]
        
        # Split data randomly in two parts
        corr_set = train_test_split(target[t].values, data_target, tid_target,
                                    train_size=0.5, shuffle=True)
        
        for i_start in range(2):

            # Prepare datasets
            target_set, data_set, tid_set = corr_set[i_start], corr_set[i_start+2], corr_set[i_start+4]
            
            # Compute correlation between data and target
            data_corr = []
            for v in range(data_set.shape[1]):
                dfocus = data_set[:, v]
                dfocus -= np.median(dfocus)
                chigh, clow = np.percentile(dfocus[dfocus>0], 99), np.percentile(dfocus[dfocus<0], 1)
                cselecter = np.logical_and(dfocus<chigh, dfocus>clow)

                data_corr.append(np.corrcoef(dfocus[cselecter], target_set[cselecter])[0, 1])
            data_corr = np.array(data_corr)
            
            # Find voxels with top target correlation
            top_sort = np.argsort(np.abs(data_corr))[::-1][:10]

            # Find voxel id of orthogonal (most uncorrelated voxel with top correlated voxel)
            orth_corr_id = np.argmin(np.abs(np.corrcoef(data_collection[:, top_sort].T)[0, :]))

            # Specify which voxels to look at
            vox_select = [top_sort[0], top_sort[orth_corr_id]]

            # Add top voxels to voxel_idx
            voxel_to_extract.append(vox_select)

    voxel_to_extract = np.unique(voxel_to_extract)
    
    # Save voxel info in CSV file
    df_out = pd.DataFrame(data_collection[:, voxel_to_extract])
    df_out.columns = ['vox_corr_idx%02d_c%02d' % (idx + 1, cix + 1) for cix in range(len(voxel_to_extract))]
    df_out.insert(0, 'Id', tid_collection)
    df_out.set_index('Id', inplace=True)
    df_out.to_csv('datasets/brain_vox_raw_train_%02d.csv' % (idx + 1))
    
    # Collect same voxel information from test set
    data_collection = []
    tid_collection = []

    for t in tqdm(test_files):

        try:
            # Load current volume and extract data values only from within the mask
            data_img = image.index_img(t, idx).get_fdata()[mask.get_fdata()>0]

            # Store data
            data_collection.append(data_img)

            # Get file name
            t_id = t.split('/')[1].split('.')[0]
            tid_collection.append(t_id)

        except:
            print(t_id)

    data_collection = np.array(data_collection)
    tid_collection = np.array(tid_collection).astype('int')

    # Save voxel info in CSV file
    df_out = pd.DataFrame(data_collection[:, voxel_to_extract])
    df_out.columns = ['vox_corr_idx%02d_c%02d' % (idx + 1, cix + 1) for cix in range(len(voxel_to_extract))]
    df_out.insert(0, 'Id', tid_collection)
    df_out.set_index('Id', inplace=True)
    df_out.to_csv('datasets/brain_vox_raw_test_%02d.csv' % (idx + 1))
    
    print('Component %02d finished.' % (idx + 1))